In [ ]:
import pandas as pd
import numpy as np
from pyteomics import mgf, mass
import itertools
from collections import Counter
import statistics
import re
import os
import glob
from scipy import stats

# 0. Initial parameters

In [ ]:
# Ion types for theoretical spectrum generation
ion_type = ['Precur', 'y', 'b', 'a', 'INT', 'IM']

# Generate each ion type
annot_yb     = True # y-, b-, a-ion
annot_precur = True # precursor ion
annot_INT    = True # internal ion
annot_IM     = True # immonium ion
annot_dict = {
    'Precur' : annot_precur,
    'y'      : annot_yb,
    'b'      : annot_yb,
    'a'      : annot_yb,
    'INT'    : annot_INT,
    'IM'     : annot_IM
}

# Mass tolerance (ppm)
ms1_ppm = 10 # MS1 level
ms2_ppm = 15 # MS2 level

# Signal-to-noise (SNR) filter for MS2 spectrum
apply_SNR = True # Apply SNR filter
SNR       = 2 # SNR threshold
low       = 0.05 # Define low x% intensity as baseline noise level

# Maximum charge state of sequence ions
max_charge = 2 # 2, 3, ... 'max'

# Maximum number of neutral loss from a single ion
max_NL = 2 # 1, 2, 3...

# Threshold for intensity coverage filter
intensity_coverage = 0.2 # Filter out spectra with sum of annotated intensities < x% of sum of total intensities

# Neutral loss from precursor & sequence ion
NL_seq = {
    'NH3' : { 'mass' : mass.calculate_mass(formula = 'NH3'),   'AA' : {'R', 'K', 'Q', 'N', 'r'} },
    'H2O' : { 'mass' : mass.calculate_mass(formula = 'H2O'),   'AA' : {'S', 'T', 'E', 'D'} },
    '43'  : { 'mass' : mass.calculate_mass(formula = 'HNCO'),  'AA' : {'r'} }, # Citrullination specific NL
    '42'  : { 'mass' : mass.calculate_mass(formula = 'CN2H2'), 'AA' : {'R'} }, # Arg specific NL
    '59'  : { 'mass' : mass.calculate_mass(formula = 'CN3H5'), 'AA' : {'R'} } # Arg specific NL
}

# Neutral loss from internal ion
NL_INT = {
    **NL_seq, 
    **{ 'CO' : { 'mass' : mass.calculate_mass(formula = 'CO'), 'AA' : {''} } } # CO loss from C-terminus of internal ion
}

# Common masses
proton = mass.calculate_mass(formula = 'H')
OH     = mass.calculate_mass(formula = 'OH')
CO     = mass.calculate_mass(formula = 'CO')
NH3    = mass.calculate_mass(formula = 'NH3')

# Symbols for common modifications
Mod_symbol = {
    'C+57.021'  : 'c', # C(Carbamidomethyl) 
    'M+15.995'  : 'm', # M(Oxidation)
    'R+0.984'   : 'r', # R(Citrullination)
    'N+0.984'   : 'n', # N(Deamidation)
    'Q+0.984'   : 'q', # Q(Deamidation)
    'E-17.027'  : '@', # Pyro-Glu from Glu
    'Q-18.011'  : '#', # Pyro-Glu from Gln    
    'K+144.102' : 'i', # K(iTRAQ 4-plex)
    'K+304.205' : 'u', # K(iTRAQ 8-plex)    
    'K+229.163' : 't', # K(TMT)
    '+144.102'  : '%', # N-term(iTRAQ 4-plex)
    '+304.205'  : '&', # N-term(iTRAQ 8-plex)
    '+229.163'  : '^', # N-term(TMT)
    '+42.011'   : 'a'  # N-term(Acetyl)
}

# N-term modification symbols
Nterm_mod_symbol = [symbol for mod, symbol in Mod_symbol.items() if len(mod.split('+')[0]) == 0]

# Amino acid monoisotopic mass
aa = {
    'c' : 160.030649, # C(Carbamidomethyl)
    'm' : 147.0354,   # M(Oxidation)
    'r' : 157.085127, # R(Citrullination)
    'n' : 115.026943, # N(Deamidation)
    'q' : 129.042594, # Q(Deamidation)
    '@' : 112.015593, # Pyro-Glu from Glu
    '#' : 110.047578, # Pyro-Glu from Gln
    'i' : 272.196963, # K(iTRAQ 4-plex)
    'u' : 432.299963, # K(iTRAQ 8-plex)
    't' : 357.257963, # K(TMT)
    '%' : 144.102,    # N-term(iTRAQ 4-plex)
    '&' : 304.205,    # N-term(iTRAQ 8-plex)
    '^' : 229.163,    # N-term(TMT)
    'a' : 42.011,     # N-term(Acetyl)
    'A' : 71.037114,
    'R' : 156.101111,
    'N' : 114.042927,
    'D' : 115.026943,
    'C' : 103.009185,
    'E' : 129.042593,
    'Q' : 128.058578,
    'G' : 57.021464,
    'H' : 137.058912,
    'I' : 113.084064,
    'L' : 113.084064,
    'K' : 128.094963,
    'M' : 131.040485,
    'F' : 147.068414,
    'P' : 97.052764,
    'S' : 87.032028,
    'T' : 101.047679,
    'W' : 186.079313,
    'Y' : 163.06332,
    'V' : 99.068414
}

# 1. Input files

In [ ]:
# Set current working directory
PATH = "F:/Project/"
os.chdir(PATH)

In [ ]:
# Input files
spec_files = glob.glob('*.mgf') # MGF file(s)
df_UnmodR  = pd.read_csv("UnmodR_peptide.csv") # UnmodR peptides w/o Cit counterpart peptides
df_total   = pd.read_csv("Total_peptide.csv") # Total identified peptides

## 1.1 Spectrum file (mgf)

In [ ]:
# Read spectrum files
dfs = []
for spec_file in spec_files:
    df_temp = pd.DataFrame(mgf.read(spec_file, convert_arrays = 1, read_charges = False))
    dfs.append(df_temp)

df_exp = pd.concat(dfs).reset_index(drop = True)
df_exp['Title'] = df_exp['params'].apply(lambda x: x.get('title')) # Get scan title
df_exp['exp_Precursor'] = df_exp['params'].apply(lambda x: x.get('pepmass')) # Get precursor m/z
df_exp['exp_Precursor'] = df_exp['exp_Precursor'].apply(lambda x: x[0])
df_exp['Charge'] = df_exp['params'].apply(lambda x: x.get('charge')) # Get precursor charge state
df_exp['Charge'] = df_exp['Charge'].apply(lambda x: x[0])
df_exp.drop(columns = ['params'], inplace = True)

In [ ]:
# Remove redundant spectra
df_exp.drop_duplicates('Title', keep = False, ignore_index = True, inplace = True)

In [ ]:
## Retain only unassigned MS/MS spectra
# Check whether each spectrum was identified by database searching
df_exp['ID'] = np.where(df_exp['Title'].isin(df_total['Title']), 1, 0)

# Retain only unassigned scans
df_unID_scan = pd.DataFrame(data = df_exp.loc[df_exp['ID'] == 0])
df_unID_scan = df_unID_scan.reset_index(drop = True)
df_unID_scan = df_unID_scan.rename(columns = {'Title': 'unID_Title'})
df_unID_scan = df_unID_scan.drop(['ID'], axis = 1)

In [ ]:
# Remove zero intensity peaks
nonzero_intensity = df_unID_scan['intensity array'].apply(lambda x: (x > 0).astype(int))
df_unID_scan['intensity array'] = (df_unID_scan['intensity array']*nonzero_intensity).apply(lambda x: [i for i in x if i != 0])
df_unID_scan['m/z array']       = (df_unID_scan['m/z array']      *nonzero_intensity).apply(lambda x: [i for i in x if i != 0])

In [ ]:
# Remove spectra with <20 peaks
df_unID_scan = df_unID_scan[df_unID_scan['m/z array'].str.len() >= 20].reset_index(drop = True)

In [ ]:
# Apply SNR filter
if apply_SNR == True:
    signal_intensity = df_unID_scan['intensity array'].apply(lambda x: 
                                                             (x > statistics.mean(sorted(x)[:round(len(x)*low)])*SNR).astype(int))
    df_unID_scan['intensity array'] = (df_unID_scan['intensity array']*signal_intensity).apply(lambda x: [i for i in x if i != 0])
    df_unID_scan['m/z array']       = (df_unID_scan['m/z array']      *signal_intensity).apply(lambda x: [i for i in x if i != 0])

## 1.2 UnmoR peptides to Cit peptides

In [ ]:
# Define a function to convert modifications to specific symbols
def Mod_to_symbol(peptide):
    for key, value in Mod_symbol.items():
        peptide = peptide.replace(key, value)
    return peptide

In [ ]:
# Define a function to remove N-term modification symbols
def Nterm_mod_remove(peptide):
    for mod in Nterm_mod_symbol:
        peptide = peptide.replace(mod, '')
    return peptide

In [ ]:
# Convert modifications to specific symbols
df_UnmodR['mod_Peptide'] = df_UnmodR['Peptide'].apply(lambda x: Mod_to_symbol(x)) # Convert modification to specific symbols

In [ ]:
# Retain only unique UnmodR peptides
df_UnmodR_pep = pd.DataFrame(df_UnmodR['mod_Peptide'])
df_UnmodR_pep = df_UnmodR_pep.drop_duplicates().reset_index(drop=True)

# Remove peptides without Arg
df_UnmodR_pep = df_UnmodR_pep[df_UnmodR_pep.mod_Peptide.str.contains("R")].reset_index(drop=True)

In [ ]:
# Count the number of Arg
df_UnmodR_pep['Arg_Count'] = df_UnmodR_pep['mod_Peptide'].str.count("R")

In [ ]:
# Sort with respect to the No. of Arg
df_UnmodR_pep = df_UnmodR_pep.sort_values('Arg_Count', ascending = True).reset_index(drop=True) 

In [ ]:
# Define a function for finding Arg position(s) on the peptide sequence
def Arg_position(sequence):    
    sequence = list(sequence)
    Arg_index = [pos + 1 for pos, AA in enumerate(sequence) if AA == 'R']
    
    return Arg_index

In [ ]:
# Find Arg position(s) on peptide sequences
df_UnmodR_pep['Arg_position'] = df_UnmodR_pep['mod_Peptide'].apply(lambda x: Arg_position(x))

In [ ]:
# Define a function for generating counterpart Cit peptide(s)
def Cit_counterpart(sequence, Arg_pos):
    appended_list = []
    for pos in Arg_pos:
        seq_list = list(sequence)
        seq_list[pos - 1] = 'r'
        seq_list = "".join(seq_list)
        appended_list.append(seq_list)
        
    return appended_list

In [ ]:
# Generate counterpart Cit peptide(s)
df_UnmodR_pep['Cit_peptide'] = df_UnmodR_pep.apply(lambda x: Cit_counterpart(x['mod_Peptide'], x['Arg_position']), axis = 1)

In [ ]:
# Define a function for calculating all possible charge states of Cit peptide
def charge_state(sequence):    
    Basic_AA = [AA for AA in sequence[0] if AA == 'R' or AA == 'K' or AA == 'H']
    max_charge = len(Basic_AA) + 1  
    charge_state = [c for c in range(1, max_charge + 1)]

    return charge_state

In [ ]:
# Create a column for possible charge states of Cit peptide
df_UnmodR_pep['Cit_Charge'] = df_UnmodR_pep['Cit_peptide'].apply(lambda x: charge_state(x))

In [ ]:
# Create a Cit dataframe
df_pep = df_UnmodR_pep[['Cit_peptide', 'Cit_Charge']]
df_pep = df_pep.rename(columns = {'Cit_peptide': 'mod_Peptide', 'Cit_Charge': 'Charge'})

In [ ]:
# Expand all Cit peptides
df_pep = df_pep.explode('mod_Peptide').reset_index(drop = True)

In [ ]:
# Expand all Cit charge states
df_pep = df_pep.explode('Charge').reset_index(drop = True)

In [ ]:
# Get Cit peptide length
df_pep['Pep_length'] = df_pep['mod_Peptide'].str.len()

In [ ]:
# Get theoretical precursor m/z
df_pep['mz_Precursor'] = ([sum(aa.get(x, 0) for x in ','.join(j)) + OH + proton 
                           for j in df_pep['mod_Peptide'].astype(str).fillna('')] + df_pep['Charge']*proton)/df_pep['Charge']

In [ ]:
# Count the number of Cit
df_pep['Cit_Count'] = df_pep['mod_Peptide'].str.count("r")

In [ ]:
# Remove N-term modification symbols
df_pep.insert(loc = 1, column = 'seq_Peptide', value = df_pep['mod_Peptide'].apply(lambda x: Nterm_mod_remove(x)))

In [ ]:
# Define a function to convert modification symbols to original delta masses
def Symbol_to_mod(peptide):
    for key, value in Mod_symbol.items():
        peptide = peptide.replace(value, key)
    return peptide

In [ ]:
# Get original peptide sequence 
df_pep.insert(loc = 0, column = 'Peptide', value = df_pep['mod_Peptide'].apply(lambda x: Symbol_to_mod(x)))

# 2. Theoretical spectrum generation

## 2.1 Ion sequence

In [ ]:
# Define functions to generate ion sequence
def getCombinations_seq (lst): # Sequence ion
    for i, j in itertools.combinations(range(len(lst) + 1), 2):
        yield lst[0:j]

def getCombinations_INT (lst): # Internal ion
    for i, j in itertools.combinations(range(len(lst) + 1), 2):
        yield lst[i:j]

In [ ]:
## Get ion sequence
# Sequence ion
df_pep_mz = df_pep.copy()

if annot_yb == True:
    # y ion
    Cterm_combo = df_pep['seq_Peptide'].str[1:].apply(lambda x: [i for i in sorted(list(set(sorted(getCombinations_seq (x[::-1]), key = len))))])
    df_pep_mz   = pd.concat([df_pep_mz, 
                             pd.DataFrame(Cterm_combo.values.tolist(), 
                                          columns = ['seq_y_' + str(i) for i in range(1, len(max(Cterm_combo, key = len)) + 1)])], axis=1)
    # b ion
    Nterm_mod = df_pep['mod_Peptide'].apply(lambda x: ''.join([AA for AA in x if AA in Nterm_mod_symbol]))
    Nterm_combo = df_pep['seq_Peptide'].str[0:-1].apply(lambda x: [i for i in sorted(list(set(sorted(getCombinations_seq (x), key = len))))])
    Nterm_combo_mod = pd.concat([Nterm_mod, Nterm_combo], axis = 1).apply(lambda x: 
                                                                          [x['mod_Peptide'] + ion for ion in x['seq_Peptide']], axis = 1)
    df_pep_mz   = pd.concat([df_pep_mz, 
                             pd.DataFrame(Nterm_combo_mod.values.tolist(), 
                                          columns = ['seq_b_' + str(i) for i in range(1, len(max(Nterm_combo_mod, key = len)) + 1)])], axis=1)
    # a ion
    df_pep_mz   = pd.concat([df_pep_mz, 
                             pd.DataFrame(Nterm_combo_mod.values.tolist(), 
                                          columns = ['seq_a_' + str(i) for i in range(1, len(max(Nterm_combo_mod, key = len)) + 1)])], axis=1)

# Internal ion
if annot_INT == True:
    INT_combo   = df_pep['seq_Peptide'].str[1:-1].apply(lambda x: [i for i in sorted(getCombinations_INT(x), key = len) if len(i) > 1])
    df_pep_mz   = pd.concat([df_pep_mz, 
                             pd.DataFrame(INT_combo.values.tolist(), 
                                          columns = ['seq_INT_' + str(i) for i in range(1, len(max(INT_combo, key = len)) + 1)])], axis=1)

In [ ]:
## Get ion annotation
# Sequence ion
df_pep_label = df_pep.copy()

if annot_yb == True:
    # y ion
    Cterm_combo_label = Cterm_combo.apply(lambda x: ['y' + str(len(i)) for i in x])
    df_pep_label = pd.concat([df_pep_label, 
                              pd.DataFrame(Cterm_combo_label.values.tolist(), 
                                           columns = ['seq_y_' + str(i) for i in range(1, len(max(Cterm_combo, key = len)) + 1)])], axis=1)
    # b ion
    Nterm_combo_label = Nterm_combo.apply(lambda x: ['b' + str(len(i)) for i in x])
    df_pep_label = pd.concat([df_pep_label, 
                              pd.DataFrame(Nterm_combo_label.values.tolist(), 
                                           columns = ['seq_b_' + str(i) for i in range(1, len(max(Nterm_combo, key = len)) + 1)])], axis=1)
    # a ion
    Nterm_combo_label = Nterm_combo.apply(lambda x: ['a' + str(len(i)) for i in x])
    df_pep_label = pd.concat([df_pep_label, 
                              pd.DataFrame(Nterm_combo_label.values.tolist(), 
                                           columns = ['seq_a_' + str(i) for i in range(1, len(max(Nterm_combo, key = len)) + 1)])], axis=1)

# Internal ion
if annot_INT == True:
    df_pep_label = pd.concat([df_pep_label, 
                              pd.DataFrame(INT_combo.values.tolist(), 
                                           columns = ['seq_INT_' + str(i) for i in range(1, len(max(INT_combo, key=len)) + 1)])], axis=1)

# Precursor ion
if annot_precur == True:
    df_pep_label.loc[(df_pep_label.mz_Precursor != np.nan), 'mz_Precursor'] = 'Precursor'

## 2.2 Singly charged ion

In [ ]:
# Define functions to calculate m/z for each ion type
def y_mz(x): # y-ion
    return sum(aa.get(a, 0) for a in ','.join(x)) + 2*(proton) + OH 

def b_mz(x): # b-ion
    return sum(aa.get(a, 0) for a in ','.join(x)) + proton

def a_mz(x): # a-ion
    return sum(aa.get(a, 0) for a in ','.join(x)) + proton - CO

def INT_mz(x): # internal ion
    return sum(aa.get(a, 0) for a in ','.join(x)) + proton

mz_dict = {
    'y'   : y_mz,
    'b'   : b_mz,
    'a'   : a_mz,
    'INT' : INT_mz
}

In [ ]:
# Calculate m/z for each ion type
for ion in ion_type:
    if ion in ['y', 'b', 'a', 'INT'] and annot_dict.get(ion) == True:
        df_temp = df_pep_mz.filter(like = 'seq_%s' % (ion)).apply(lambda x: [mz_dict.get(ion)(i) if pd.notnull(i) else np.nan for i in x])
        df_temp.rename(columns = lambda x: x.replace('seq', 'mz'), inplace = True)
        df_pep_mz = pd.concat([df_pep_mz, df_temp], axis = 1)

# Update annotation dataframe
df_label_temp = df_pep_label.filter(like='seq').drop(columns = 'seq_Peptide')
df_label_temp.rename(columns = lambda x: x.replace('seq', 'mz'), inplace = True)
df_pep_label = pd.concat([df_pep_label, df_label_temp], axis = 1)
df_pep_label.fillna(value = np.nan, inplace = True)

# Replace None to NaN
df_pep_mz.fillna(value = np.nan, inplace = True)
df_pep_label.fillna(value = np.nan, inplace = True)

In [ ]:
# Drop empty columns
df_pep_mz    = df_pep_mz.dropna(how = 'all', axis = 1)
df_pep_label = df_pep_label.dropna(how = 'all', axis = 1)

## 2.3 Neutral loss variant

In [ ]:
# Define a function for generating NL combinations
def NL_combo(x, max_NL):    
    
    NL_label = [] # NL combinations
    for i in range(1, max_NL + 1):
        temp = list(itertools.product(list(x.keys()),repeat = i))
        NL_label.append(temp)
        
    NL_label = list(itertools.chain.from_iterable(NL_label))
    NL_label = [tuple(sorted(tuple_)) for tuple_ in NL_label]
    myset = set(NL_label)
    NL_label = sorted(myset)
    for i in range(len(NL_label)):
        NL_label[i] = '-'.join(NL_label[i])    
        
    NL_mz = [sub.split('-') for sub in NL_label] # NL combinations m/z
    for i in range(len(NL_mz)):
        NL_mz[i] = sum(x.get(NL_mz[i][j])['mass'] for j in range(len(NL_mz[i]))) 
        
    return NL_label, NL_mz

In [ ]:
# Define a function for checking whether NL condition is met for each ion
def NL_condition(NL_AA_list, sequence):    
    
    c = Counter(sequence)
    
    # If one-to-one NL type exists, take care of it first
    for NL_group in NL_AA_list:
        if len(NL_group) == 1:
            if NL_group[0] not in sequence:
                return
            else:
                if len(NL_AA_list) > 1:
                    if c[NL_group[0]] >= 1:
                        c[NL_group[0]] -= 1
                    elif c[NL_group[0]] <= 0:
                        return
                elif len(NL_AA_list) == 1:
                    return True
    
    # Check if remaining NL types is feasible in each ion
    NL_AA_list = [NL_group for NL_group in NL_AA_list if len(NL_group) > 1]
    common_AA  = list(set(itertools.chain.from_iterable(NL_AA_list)))
    
    # If there are only one-to-one NL types, return True
    if len(NL_AA_list) == 0:
        return True
    
    # In case of one-to-multiple NL types
    for AA in list(c):
        if AA not in common_AA:
            del c[AA]
        
    if sum(c.values()) >= len(NL_AA_list):
        return True   

In [ ]:
# Create NL variant dataframes
df_pep_mz_temp    = df_pep_mz.copy()
df_pep_label_temp = df_pep_label.copy()

for ion in ['Precur', 'y', 'b', 'a', 'INT']:
    if annot_dict.get(ion) == True:
        NL_list = []
        if ion == 'Precur':
            NL_list  = NL_combo(NL_seq, max_NL)
            df_seq   = df_pep_mz_temp[['seq_Peptide']]
            df_mz    = df_pep_mz_temp[['mz_Precursor']]
            df_label = df_pep_label_temp[['mz_Precursor']]
        elif ion in ['y', 'b', 'a']:
            NL_list     = NL_combo(NL_seq, max_NL)
            df_seq      = df_pep_mz_temp[[col for col in df_pep_mz_temp.columns if 'seq_%s' %(ion) in col]]
            df_seq_temp = pd.Series(df_seq.values.tolist()).apply(lambda x: [i for i in x if isinstance(i, str)])
            df_mz       = df_pep_mz_temp[[col for col in df_pep_mz_temp.columns if 'mz_%s'  %(ion) in col]]
            df_label    = df_pep_label_temp[[col for col in df_pep_label_temp.columns if 'mz_%s' %(ion) in col]]       
        elif ion == 'INT':
            NL_list     = NL_combo(NL_INT, max_NL)
            df_seq      = df_pep_mz_temp[[col for col in df_pep_mz_temp.columns if 'seq_%s' %(ion) in col]]
            df_seq_temp = pd.Series(df_seq.values.tolist()).apply(lambda x: [i for i in x if isinstance(i, str)])
            df_mz       = df_pep_mz_temp[[col for col in df_pep_mz_temp.columns if 'mz_%s'  %(ion) in col]]
            df_label    = df_pep_label_temp[[col for col in df_pep_mz_temp.columns if 'mz_%s' %(ion) in col]]

        # For each NL type get NL variant m/z
        if NL_list != []: # If the ion types are to be included
            for NL in NL_list[0]:
                NL_AA_list = []
                for sub_NL in NL.split('-'):

                    if ion in ['Precur', 'y', 'b', 'a']:
                        temp = list(NL_seq.get(sub_NL)['AA'])
                    elif ion == 'INT':
                        temp = list(NL_INT.get(sub_NL)['AA'])
                    NL_AA_list.append(temp)
                
                # m/z dataframe
                if ion == 'Precur':
                    df_bool     = df_seq.applymap(lambda x: 1 if NL_condition(NL_AA_list, str(x)) == True else np.nan)
                    df_mz_temp  = pd.DataFrame(df_bool.values*df_mz.values, columns = df_mz.columns, index = df_mz.index)
                    df_mz_temp  = df_mz_temp.replace({0:np.nan})
                    df_mz_temp['mz_Precursor'] = df_mz_temp.mz_Precursor*df_pep_mz_temp.Charge # Convert m/z to mass
                    df_mz_temp  = df_mz_temp.apply(lambda x: 0 if str(x) == np.nan else x - NL_list[1][NL_list[0].index(NL)])
                    df_mz_temp['mz_Precursor'] = df_mz_temp.mz_Precursor/df_pep_mz_temp.Charge # Convert mass to m/z
                    df_mz_temp.rename(columns = lambda x: x + '-%s' % (NL), inplace = True)
                    df_pep_mz_temp = pd.concat([df_pep_mz_temp, df_mz_temp], axis = 1)                
                else:
                    df_bool     = pd.DataFrame(df_seq_temp.apply(lambda x: [1 if NL_condition(NL_AA_list, i) == True 
                                                                            else np.nan for i in x]).tolist())
                    df_mz_temp  = pd.DataFrame(df_bool.values*df_mz.values, columns = df_mz.columns, index = df_mz.index)
                    NL_mz       = NL_list[1][NL_list[0].index(NL)]
                    df_mz_temp  = df_mz_temp - NL_mz
                    df_mz_temp.rename(columns = lambda x: x + '-%s' % (NL), inplace = True)
                    df_pep_mz_temp = pd.concat([df_pep_mz_temp, df_mz_temp], axis = 1)

                # Annotation dataframe
                df_label_temp = (df_bool==1).astype(int) * df_label.values
                df_label_temp = df_label_temp.replace(r'^\s*$', np.nan, regex=True)
                df_label_temp.columns = df_label.columns
                df_label_temp = pd.DataFrame(np.where(df_label_temp.notnull(), df_label_temp.astype(str) + '-' + NL, np.nan), 
                                             columns = df_label_temp.columns)
                df_label_temp.rename(columns = lambda x: x + '-%s' % (NL), inplace = True)
                df_pep_label_temp = pd.concat([df_pep_label_temp, df_label_temp], axis = 1)

In [ ]:
# Copy temporary dataframe back into original dataframe
df_pep_mz    = df_pep_mz_temp.copy()
df_pep_label = df_pep_label_temp.copy()

In [ ]:
# Drop empty columns
df_pep_mz    = df_pep_mz.dropna(how = 'all', axis = 1)
df_pep_label = df_pep_label.dropna(how = 'all', axis = 1)

## 2.4 Multiply charged ion

In [ ]:
# Define a function for checking whether multiple charge state condition is met for each ion
def multi_charge_condition(charge, sequence):
    
    if pd.notnull(sequence):
        # Count the number of basic residues
        AA_count = sequence.count('R') + sequence.count('K') + sequence.count('H')

        # Condition for multiple charge state
        if AA_count + 1 >= charge:
            return 1

In [ ]:
# Create multiple charge state dataframe
if max_charge > 1:
    # Create temporary m/z and annotation dataframe
    df_seq   = df_pep_mz[[col for col in df_pep_mz.columns       
                          if any(['seq_y' in col, 'seq_b' in col, 'seq_a' in col]) == True]]
    df_mz    = df_pep_mz[[col for col in df_pep_mz.columns       
                          if any(['mz_y' in col, 'mz_b' in col, 'mz_a' in col]) == True]]
    df_label = df_pep_label[[col for col in df_pep_label.columns 
                          if any(['mz_y' in col, 'mz_b' in col, 'mz_a' in col]) == True]]
    df_pep_mz_temp    = df_pep_mz.copy()
    df_pep_label_temp = df_pep_label.copy()

    # For each charge state get m/z
    if max_charge == 'max':
        charge_boundary = df_pep_mz['Charge'].max()
    else:
        charge_boundary = max_charge

    for charge in range(2, charge_boundary + 1): # Generate multiply charged sequence ions (from +2 to predefined max. charge)
        # m/z dataframe
        df_charge = df_pep_mz['Charge'].copy()
        df_charge.loc[df_charge < charge] = 0
        df_bool1  = df_seq.apply(lambda x: [multi_charge_condition(charge, ion) if isinstance(ion, str) else 0 for ion in x])
        df_bool1  = pd.concat([df_charge, df_bool1], axis = 1)
        df_bool1.loc[df_bool1['Charge'] == 0, df_bool1.columns] = 0
        df_bool1  = df_bool1.drop(columns = 'Charge')

        df_mz_temp  = df_mz.copy()
        for column in df_mz_temp.columns:
            df_mz_temp[column] = df_bool1['%s' % (column.split('-')[0].replace('mz', 'seq'))]
        df_mz_temp  = df_mz_temp.replace({0:np.nan})
        df_mz_temp  = pd.DataFrame(df_mz_temp.values*df_mz.values, columns = df_mz_temp.columns, index = df_mz_temp.index)
        df_mz_temp  = (df_mz_temp + proton*(charge - 1))/charge
        df_mz_temp.rename(columns = lambda x: x + '%s' % ("+"*charge ), inplace = True)
        df_pep_mz_temp = pd.concat([df_pep_mz_temp, df_mz_temp], axis = 1)

        # Annotation dataframe
        df_bool2      = df_mz_temp.notnull().astype(int)  
        df_label_temp = pd.DataFrame(df_bool2.values*df_label.values,
                                     columns = df_label.columns, index = df_label.index)
        df_label_temp = df_label_temp.replace(r'^\s*$', np.nan, regex=True)
        df_label_temp = df_label_temp.replace({0:np.nan})
        df_label_temp = pd.DataFrame(np.where(df_label_temp.notnull(), df_label_temp.astype(str) + '+'*charge, np.nan), 
                                     columns = df_label_temp.columns)
        df_label_temp.columns = df_label_temp.columns + '+'*charge
        df_pep_label_temp = pd.concat([df_pep_label_temp, df_label_temp], axis = 1)

In [ ]:
# Copy temporary dataframe back into original dataframe
df_pep_mz    = df_pep_mz_temp.copy()
df_pep_label = df_pep_label_temp.copy()

## 2.5 Immonium ion

In [ ]:
# Immonium ions (IM, IM-NH3)
if annot_IM == True:
    # m/z dataframe
    df_IM_mz             = pd.DataFrame([[aa.get(AA) + proton - CO for AA in seq] for seq in df_pep_mz.seq_Peptide.tolist()])
    df_IM_mz.columns     = ['mz_IM_%s' % (n+1) for n in range(df_pep_mz['Pep_length'].max())]    
    df_IM_NH3_mz         = df_IM_mz - NH3
    df_IM_NH3_mz.columns = [col.replace('IM', 'IM-NH3') for col in df_IM_mz.columns]
    df_pep_mz            = df_pep_mz.join([df_IM_mz, df_IM_NH3_mz])

    # Annotation dataframe
    df_IM_label             = pd.DataFrame([['IM(%s)' % (AA) for AA in seq] for seq in df_pep_label.seq_Peptide.tolist()])
    df_IM_label.columns     = ['mz_IM_%s' % (n+1) for n in range( df_pep_label['Pep_length'].max())]   
    df_IM_NH3_label         = pd.DataFrame(np.where((df_IM_label.notnull()), df_IM_label.astype(str) + '-NH3', np.nan))
    df_IM_NH3_label.columns = [col.replace('IM', 'IM-NH3') for col in df_IM_label.columns]
    df_pep_label            = df_pep_label.join([df_IM_label, df_IM_NH3_label])

In [ ]:
# Drop empty columns
df_pep_mz    = df_pep_mz.dropna(how = 'all', axis = 1)
df_pep_label = df_pep_label.dropna(how = 'all', axis = 1)

# 3. Match precursor m/z & charge between experimental & theoretical spectra

In [ ]:
# Sort dataframes based on precursor m/z
new_row_index = list(df_pep_mz.sort_values('mz_Precursor', ascending = True).index)
df_pep_mz.sort_values('mz_Precursor', ascending = True, ignore_index = True, inplace = True)
df_pep_label  = df_pep_label.reindex(new_row_index).reset_index(drop = True)
df_unID_scan.sort_values('exp_Precursor', ascending = True, ignore_index = True, inplace = True)

In [ ]:
# Get precursor m/z of theoretical and unassigned spectra
mz_theo = pd.DataFrame(df_pep_mz['mz_Precursor'])
mz_unID = pd.DataFrame(df_unID_scan['exp_Precursor'])

In [ ]:
## Check whether precursor m/z matches between theoretical and unassigned spectra
# Split unassigned dataframe into smaller chunks
chunk = 10
mz_unID_chunk       = np.array_split(mz_unID, chunk)
appended_unID       = pd.DataFrame()
appended_theo       = pd.DataFrame()
appended_unID_order = pd.DataFrame()

for i in range(0, chunk):
    
    # Get theoretical precursor m/z in unassigned chunk m/z range
    mz_theo_slice = mz_theo[mz_theo['mz_Precursor'].between(mz_unID_chunk[i].iloc[0][0], mz_unID_chunk[i].iloc[-1][0])]
    
    # Get nominal m/z
    theo_array     = np.array(mz_theo_slice)
    unID_array     = np.array(mz_unID_chunk[i])
    theo_array_int = theo_array.astype(int)
    unID_array_int = unID_array.astype(int)
    
    # Check for matching nominal m/z
    theo_array_overlap  = np.in1d(theo_array_int, unID_array_int).reshape(len(theo_array_int), 1)*theo_array
    theo_array          = theo_array_overlap[theo_array_overlap > 0] # Retain only nominal m/z-matched m/z
    unID_array_overlap  = np.in1d(unID_array_int, theo_array_int).reshape(len(unID_array_int), 1)*unID_array
    unID_array          = unID_array_overlap[unID_array_overlap > 0] # Retain only nominal m/z-matched m/z
    unID_order          = (unID_array_overlap > 0).astype(int) # Keep original unassigned array order
    theo_order          = (theo_array_overlap > 0).astype(int)  # Keep original theoretical array order
    appended_unID_order = pd.concat([appended_unID_order, pd.DataFrame(unID_order)], axis = 0)
    
    # Match precursor m/z
    array_mz_theo   = np.tile(theo_array.reshape(len(theo_array), 1), (1, unID_array.shape[0]))
    array_mz_unID   = np.transpose(np.tile(unID_array.reshape(len(unID_array), 1), (1, theo_array.shape[0])))
    array_mz_eCheck = abs(array_mz_theo - array_mz_unID) < (array_mz_theo*ms1_ppm*1e-6)
    array_match     = 1*(array_mz_eCheck.sum(axis = 0) > 0)
    appended_unID   = pd.concat([appended_unID, pd.DataFrame(array_match)], axis = 0)

    # Attach matched theoretical spectra indices
    df_theo_order       = pd.DataFrame(theo_order)
    df_theo_order.index = mz_theo_slice.index
    df_check            = pd.DataFrame(array_mz_eCheck)
    theo_match_index    = df_theo_order.index[df_theo_order[0] != 0].tolist()
    df_check.index      = theo_match_index
    empty_df            = pd.DataFrame(index = df_theo_order.index, columns = [0])
    df_check            = empty_df.combine_first(df_check)
    
    appended_theo_temp  = []
    for col in df_check.columns:
        list_temp = df_check.index[df_check[col] == True].tolist()
        appended_theo_temp.append(list_temp)
    appended_theo = pd.concat([appended_theo, pd.DataFrame(appended_theo_temp)], axis = 0)
    print('Matching precursor m/z btw theoretical & unassigned spectra: ' + str(i) + str('/%s chunks processed') %(chunk), end="\r")

appended_unID       = appended_unID.reset_index(drop = True)
appended_theo       = appended_theo.reset_index(drop = True)
appended_unID_order = appended_unID_order.reset_index(drop=True)

In [ ]:
# Reshape to original dataframe shape
unID_match_index    = appended_unID_order.index[appended_unID_order[0] != 0].tolist() # Get matched row indices
appended_unID.index = unID_match_index # Match with original row index
appended_theo.index = unID_match_index # Match with original row index
empty_df            = pd.DataFrame(index = appended_unID_order.index, columns = [0])
appended_unID_match = empty_df.combine_first(appended_unID) # Reshape to original row length
appended_theo_match = empty_df.combine_first(appended_theo) # Reshape to original row length

In [ ]:
# Merge with experimental dataframe
df_unID_scan['mz_Check']              = appended_unID_match
df_unID_scan['matched_theo_spectrum'] = appended_theo_match.values.tolist()

In [ ]:
# Create a filtered experimental dataframe
df_unID_scan_filter_mz = df_unID_scan.loc[df_unID_scan['mz_Check'] == 1].reset_index(drop = True) # Retain only precursor m/z matched spectra
df_unID_scan_filter_mz['matched_theo_spectrum'] = df_unID_scan_filter_mz['matched_theo_spectrum'].apply(lambda x: [int(index) for index in x if pd.notnull(index)])
df_unID_scan_filter_mz = df_unID_scan_filter_mz.explode('matched_theo_spectrum').reset_index(drop = True) # Expand all rows with multiple matching theo spectra

In [ ]:
# Check for charge match
charge_theo = pd.DataFrame(df_pep_mz['Charge']).rename(columns = {'Charge': 'Charge_theo'})
charge_theo['matched_theo_spectrum'] = charge_theo.index
df_unID_scan_filter_mz = df_unID_scan_filter_mz.merge(charge_theo, on = 'matched_theo_spectrum')

In [ ]:
# Create a m/z-charge filtered unassigned dataframe
df_unID_scan_filter_mz['Charge_Check'] = df_unID_scan_filter_mz[['Charge', 'Charge_theo']].apply(lambda x: 
                                                                                                 1 if x['Charge'] == x['Charge_theo'] 
                                                                                                 else 0, axis = 1)
df_unID_scan_filter_mz_charge = df_unID_scan_filter_mz[df_unID_scan_filter_mz['Charge_Check'] == 1].reset_index(drop = True)
df_unID_scan_filter_mz_charge.drop(columns = ['mz_Check', 'Charge_theo', 'Charge_Check'], inplace = True)

In [ ]:
## Retain only theoretical spectra with precursor m/z & charge matched to those of unassigned spectra
# m/z dataframe
df_pep_mz_filter = df_pep_mz.loc[list(set([i for i in df_unID_scan_filter_mz_charge['matched_theo_spectrum'].tolist()]))]
df_pep_mz_filter.insert(loc = 0, column = 'matched_theo_spectrum', value = df_pep_mz_filter.index)
df_pep_mz_filter = df_pep_mz_filter.reset_index(drop = True)
df_pep_mz_filter.drop(columns = ['mz_Precursor', 'Charge'], inplace = True)

# Annotation dataframe
df_pep_label_filter = df_pep_label.loc[list(set([i for i in df_unID_scan_filter_mz_charge['matched_theo_spectrum'].tolist()]))]
df_pep_label_filter.insert(loc = 0, column = 'matched_theo_spectrum', value = df_pep_label_filter.index)
df_pep_label_filter = df_pep_label_filter.reset_index(drop = True)
df_pep_label_filter.drop(columns = ['mz_Precursor', 'Charge'], inplace = True)

# 4. XCorr score calculation

In [ ]:
# Define a function to get binned spectrum
def bin_spec(mz_array, intensity_array):
    # Set first & end mass for binning MS2 spectra
    first_mass = 140
    end_mass   = 1000
    fragment_bin_tol = 0.01 # Set fragment bin tolerance (Da)
    bin_size         = int((end_mass - first_mass)/fragment_bin_tol) # Set bin size
    
    binned_spectrum = stats.binned_statistic(mz_array, intensity_array, 
                                             statistic = 'max', bins = bin_size, range = (first_mass, end_mass))[0]
    
    return binned_spectrum

In [ ]:
# Define a function to get binned theoretical spectrum
def bin_theo_spec(mz_array):
    # Set first & end mass for binning MS2 spectra
    first_mass = 140
    end_mass   = 1000
    fragment_bin_tol = 0.01 # Set fragment bin tolerance (Da)
    bin_size         = int((end_mass - first_mass)/fragment_bin_tol) # Set bin size
    
    # Set all intensities of theroetical peaks to 1
    intensity_array = np.empty(len(mz_array))
    intensity_array.fill(1)
    
    binned_spectrum = stats.binned_statistic(mz_array, intensity_array, 
                                             statistic = 'max', bins = bin_size, range = (first_mass, end_mass))[0]
    
    return binned_spectrum

In [ ]:
# Sort by matched spectrum index
df_unID_scan_filter_mz_charge.sort_values('matched_theo_spectrum', ignore_index = True, inplace = True)
df_pep_mz_filter.sort_values('matched_theo_spectrum', ignore_index = True, inplace = True)
df_pep_label_filter.sort_values('matched_theo_spectrum', ignore_index = True, inplace = True)

In [ ]:
# Add a m/z array column to theoretical dataframe
df_pep_mz_filter['m/z array'] = df_pep_mz_filter.filter(like = 'mz').values.tolist()
df_pep_mz_filter['m/z array'] = df_pep_mz_filter['m/z array'].apply(lambda x: [mz for mz in x if pd.notnull(mz)])

In [ ]:
# Split unassigned dataframe into smaller chuncks
unID_match_index    = list(set(df_unID_scan_filter_mz_charge['matched_theo_spectrum'].tolist()))
appended_Xcorr_unID = pd.DataFrame()

for i in unID_match_index: # Split unassigned dataframe by same matched spectrum index
        
    # Spectrum binning of unassigned spectra
    df_unID_chunk     = df_unID_scan_filter_mz_charge[df_unID_scan_filter_mz_charge['matched_theo_spectrum'] == i]
    binned_unID_array = np.vstack(df_unID_chunk[['m/z array', 'intensity array']].apply(lambda x: bin_spec(x['m/z array'], x['intensity array']), axis = 1))
    
    # Square root transformation & base peak normalization of unassigned spectra
    binned_unID_array_sqrt_norm = np.sqrt(binned_unID_array)/np.nanmax(np.sqrt(binned_unID_array), axis = 1)[:,None] 
    
    # Spectrum binning of theoretical spectra
    df_pep_mz_filter_temp  = df_pep_mz_filter[df_pep_mz_filter['matched_theo_spectrum'] == i].reset_index(drop = True)
    binned_pep_array       = np.vstack(df_pep_mz_filter_temp.apply(lambda x: bin_theo_spec(x['m/z array']), axis = 1))
    
    # Square root transformation & base peak normalization of theoretical spectra
    binned_pep_array_sqrt_norm      = np.sqrt(binned_pep_array)/np.nanmax(np.sqrt(binned_pep_array), axis = 1)[:,None]
    binned_pep_array_sqrt_norm_tile = np.tile(binned_pep_array_sqrt_norm, 
                                              (binned_unID_array_sqrt_norm.shape[0], 1)) # Reshape to unassigned array shape
    
    # Drop all NaN-only columns common to both unassigned & theoretical spectra
    arr_concat                      = np.concatenate([binned_unID_array_sqrt_norm, binned_pep_array_sqrt_norm_tile], axis = 0)
    arr_concat_filter               = arr_concat[:,~np.all(np.isnan(arr_concat), axis = 0)]
    arr_concat_filter[np.isnan(arr_concat_filter)] = 0
    binned_unID_array_sqrt_norm     = arr_concat_filter[0:binned_unID_array_sqrt_norm.shape[0],:]
    binned_pep_array_sqrt_norm_tile = arr_concat_filter[binned_unID_array_sqrt_norm.shape[0]:,:]
    
    # Sum all intensities for m/z shifts
    unID_array = np.zeros(shape = binned_unID_array_sqrt_norm.shape)
    
    for j in range(-75, 76): 
        if j != 0:
            if j < 0:
                binned_unID_array_sqrt_norm_shift          = np.roll(binned_unID_array_sqrt_norm, j, axis = 1)
                binned_unID_array_sqrt_norm_shift[:,-j:]   = 0
                unID_array += binned_unID_array_sqrt_norm_shift
            elif j > 0:
                binned_unID_array_sqrt_norm_shift          = np.roll(binned_unID_array_sqrt_norm, j, axis = 1)
                binned_unID_array_sqrt_norm_shift[:,0:0+j] = 0
                unID_array += binned_unID_array_sqrt_norm_shift
    
    # Calculate Xcorr score
    Xcorr_avg           = np.sum(binned_pep_array_sqrt_norm_tile*unID_array, axis = 1)/(2*75)
    Xcorr_score         = np.sum(binned_pep_array_sqrt_norm_tile*binned_unID_array_sqrt_norm, axis = 1) - Xcorr_avg
    appended_Xcorr_unID = pd.concat([appended_Xcorr_unID, pd.DataFrame(Xcorr_score)], axis = 0)
    print('Unassigned Xcorr score calculation: %s/%s chunks processed' %(unID_match_index.index(i), len(unID_match_index)), end="\r")  

In [ ]:
# Append Xcorr column to unassigned dataframe
appended_Xcorr_unID = appended_Xcorr_unID.reset_index(drop = True)
df_unID_scan_filter_mz_charge['Xcorr'] = appended_Xcorr_unID

In [ ]:
# Groupby unassigned scan title
info_col      = ['unID_Title', 'exp_Precursor', 'Charge']
df_unID_info  = df_unID_scan_filter_mz_charge[info_col].groupby('unID_Title', as_index = False).agg(lambda x: list(set(list(x)))[0])
spec_col      = ['unID_Title', 'm/z array', 'intensity array', 'matched_theo_spectrum']
df_unID_spec  = df_unID_scan_filter_mz_charge[spec_col].groupby('unID_Title', as_index = False).agg(list)
df_unID_Xcorr = df_unID_scan_filter_mz_charge[['unID_Title', 'Xcorr']].groupby('unID_Title', as_index = False).agg(list)
df_unID_group = df_unID_info.merge(df_unID_spec, on = 'unID_Title').merge(df_unID_Xcorr, on = 'unID_Title')

In [ ]:
# Get the highest Xcorr for each unassigned spectrum
df_unID_group['Best_Xcorr'] = df_unID_group['Xcorr'].apply(max)

In [ ]:
# Get the mathced theoretical spectrum index with the highest Xcorr score
df_unID_group['Best_matched_theo_spectrum'] = df_unID_group[['matched_theo_spectrum', 'Xcorr']].apply(lambda x: 
                                                        x['matched_theo_spectrum'][x['Xcorr'].index(max(x['Xcorr']))], axis = 1)

In [ ]:
# Get the mathced theoretical spectrum m/z array with the highest Xcorr score
df_unID_group['m/z array'] = df_unID_group[['m/z array', 'Xcorr']].apply(lambda x: 
                                                                   x['m/z array'][x['Xcorr'].index(max(x['Xcorr']))], 
                                                                   axis = 1)

In [ ]:
# Get the mathced theoretical spectrum intensity array with the highest Xcorr score
df_unID_group['intensity array'] = df_unID_group[['intensity array', 'Xcorr']].apply(lambda x: 
                                                                               x['intensity array'][x['Xcorr'].index(max(x['Xcorr']))], 
                                                                               axis = 1)

In [ ]:
# Sort experimental dataframe according to theoretical spectrum indices
df_unID_group = df_unID_group.sort_values('Best_matched_theo_spectrum', ascending = True).reset_index(drop=True)

In [ ]:
## m/z dataframe
# Create a new theoretical m/z dataframe
df_pep_mz_info  = df_pep_mz_filter[['Peptide', 'mod_Peptide', 'Pep_length', 'Cit_Count']]
df_pep_mz_sub   = df_pep_mz_info.join(df_pep_mz_filter[['matched_theo_spectrum', 'm/z array']])
df_pep_mz_match = pd.merge(df_pep_mz_sub, df_unID_group['Best_matched_theo_spectrum'], 
                           left_on = ['matched_theo_spectrum'], right_on = ['Best_matched_theo_spectrum'], how = 'right')
df_pep_mz_match = df_pep_mz_match.sort_values('Best_matched_theo_spectrum', ascending = True).reset_index(drop=True)

In [ ]:
## Annotation dataframe
# Expand theoretical annotation dataframe
df_pep_label_filter['label array'] = df_pep_label_filter.filter(like='mz').values.tolist()
df_pep_label_filter['label array'] = df_pep_label_filter['label array'].apply(lambda x: 
                                                                              [label for label in x if pd.notnull(label)])

# Create a new theoretical annotation dataframe
df_pep_label_info  = df_pep_label_filter[['Peptide', 'mod_Peptide', 'Pep_length', 'Cit_Count']]
df_pep_label_sub   = df_pep_label_info.join(df_pep_label_filter[['matched_theo_spectrum', 'label array']])
df_pep_label_match = pd.merge(df_pep_label_sub, df_unID_group['Best_matched_theo_spectrum'], 
                              left_on = ['matched_theo_spectrum'], right_on = ['Best_matched_theo_spectrum'], how = 'right')
df_pep_label_match = df_pep_label_match.sort_values('Best_matched_theo_spectrum', ascending = True).reset_index(drop=True)

# 5. Match experimental with theoretical peaks

In [ ]:
# Create a column of mz-intensity pairs
df_unID_group['mz_intensity'] = df_unID_group[['m/z array', 'intensity array']].apply(lambda x: 
                                                                                    [list(t) for t in zip(x['m/z array'], 
                                                                                                          x['intensity array'])], axis = 1)

In [ ]:
# Experimental spectrum
df_exp_temp  = df_unID_group.copy()
df_exp_temp2 = pd.DataFrame(df_exp_temp['mz_intensity'].values.tolist())
df_exp_temp2.fillna(value = np.nan, inplace = True)
df_exp_mz    = df_exp_temp['m/z array'].copy()

# Theoretical spectrum
df_theo_mz    = pd.DataFrame(df_pep_mz_match['m/z array'].values.tolist()).copy() # Theoretical ion m/z
df_theo_label = pd.DataFrame(df_pep_label_match['label array'].values.tolist()).copy() # Theoretical ion annotation

In [ ]:
# For each Experimental spectrum, compare experimental vs. theoretical spectrum and retain only matching ions
appended_mz    = []
appended_label = []
appended_order = []

for r in range(len(df_exp_mz)):
    # Get non-NaN peaks only
    exp_array  = np.array(df_exp_mz[r]) # Experimental peaks
    exp_array  = exp_array[~np.isnan(exp_array)] 
    theo_array = np.array(df_theo_mz.loc[r]) # Theoretical peaks
    theo_array = theo_array[~np.isnan(theo_array)]
    
    # Initial m/z match using nominal m/z
    exp_array_int      = exp_array.astype(int)  # Get nominal experimental m/z
    theo_array_int     = theo_array.astype(int) # Get nominal theoretical m/z
    exp_array_overlap  = np.in1d(exp_array_int, theo_array_int) * exp_array # Check only overlapping nominal m/z
    exp_array          = exp_array_overlap[exp_array_overlap > 0] # Retain only overlapping experimental m/z
    theo_array_overlap = np.in1d(theo_array_int, exp_array_int) * theo_array # Check only overlapping nominal m/z
    theo_array         = theo_array_overlap[theo_array_overlap > 0] # Retain only overlapping theoretical m/z
    exp_order          = (exp_array_overlap > 0).astype(int) # Keep original array order
    appended_order.append(exp_order)
    
    # Final m/z match within a predefined m/z tolerance
    array_exp_mz  = np.tile(exp_array, (theo_array.shape[0], 1)) # Experimental peak array
    array_theo_mz = np.transpose(np.tile(theo_array, (exp_array.shape[0], 1))) # Theoretical peak array
    array_eCheck  = abs(array_theo_mz - array_exp_mz) < (array_theo_mz*ms2_ppm * 1e-6) 
    array_match   = 1*(array_eCheck.sum(axis = 0) > 0)
    appended_mz.append(array_match)
    
    # Get annotation for the retained experimental peaks
    exp_mz          = np.sum((array_exp_mz*array_eCheck), axis = 1)
    theo_label      = np.array(df_theo_label.loc[r].dropna())
    theo_ex_overlap = (theo_label * (theo_array_overlap > 0).astype(bool))
    theo_label      = np.array([x for x in theo_ex_overlap if x])
    
    # Merge peak m/z and annotation
    mz_label_pair = pd.DataFrame(np.vstack((exp_mz, theo_label))).T
    mz_label_pair.columns = ["exp_mz", "label"]
    mz_label_pair['exp_mz'] = mz_label_pair['exp_mz'].astype(float)
    mz_label_pair.loc[mz_label_pair['exp_mz'] == 0, 'label'] = np.nan
    mz_label_pair_group = mz_label_pair.groupby('exp_mz', as_index = False)['label'].agg({'labels':(lambda x: list(set(x)))})
    mz_label_pair_group = mz_label_pair_group[mz_label_pair_group['exp_mz'] != 0]
    appended_label.append(np.array(mz_label_pair_group['labels']))
    print(str(r + 1) + str('/%s spectra processed') %(len(df_exp_mz)), end = "\r")

In [ ]:
# Retain only nominal m/z-matched experimental peaks
all_data_order     = (pd.DataFrame(appended_order) > 0).astype(int)
df_exp_temp2_order = (df_exp_temp2*all_data_order).applymap(lambda x: np.nan if x == [] else x) 
df_exp_temp2_order = df_exp_temp2_order.apply(lambda row: pd.Series(row.dropna().values), axis = 1)

# Retain only m/z matched experimental peaks
all_data_mz = (pd.DataFrame(appended_mz) > 0).astype(int)
df_obs      = pd.DataFrame(df_exp_temp2_order.values*all_data_mz.values, 
                      columns = all_data_mz.columns, index = all_data_mz.index)
df_obs      = df_obs.applymap(lambda x: np.nan if x == [] else x)
df_obs      = df_obs.apply(lambda row: pd.Series(row.dropna().values), axis = 1)

# Merge m/z-intensity-annotation
all_data_label = pd.DataFrame([df for df in appended_label])
all_data_label = all_data_label.reset_index(drop = True)
all_data_label.columns = range(all_data_label.shape[1])
df_mz_label    = (df_obs + all_data_label).dropna(how = 'all', axis = 1)
df_mz_label.columns = ['peak_%s' % (i+1) for i in range(len(df_mz_label.columns))] # Rename columns

In [ ]:
# Append peptide info columns
df_mz_label = df_exp_temp[['unID_Title', 'exp_Precursor', 'Charge']].join(df_pep_mz_match[['Peptide', 'mod_Peptide', 
                                                                            'Pep_length', 'Cit_Count']]).join(df_mz_label)

# 6. Cit diagnostic ion analysis

In [ ]:
# Make a new dataframe
df_mz_label_uniq = df_mz_label.copy()

In [ ]:
# Define a function to filter out secondary ions w/o primary ions (for precursor & sequence ion only)
def Secondary_filter(Primary_ion_list, Secondary_ion_list):
    # Get secondary ion of precursor & sequence ion
    Secondary_ion_pri  = [re.sub(r'[0-9]+', '', i.split('-')[0]) for i in Secondary_ion_list]
    Secondary_ion_precur_seq = [a*b for a, b in zip([True if ion in ['Precursor', 'y', 'b', 'a', 'z', 'c'] else False 
                                                     for ion in Secondary_ion_pri], Secondary_ion_list)]
    Secondary_ion_precur_seq = [ion for ion in Secondary_ion_precur_seq if len(ion) > 0]
    
    Secondary_ion_list_filtered = []
    for sec in Secondary_ion_list:
        if sec in Secondary_ion_precur_seq:
            if sec.split('-')[0] in Primary_ion_list:
                Secondary_ion_list_filtered.append(sec)
        else:
            Secondary_ion_list_filtered.append(sec)
    
    return Secondary_ion_list_filtered

In [ ]:
# Filter out secondary ions without primary ions (precursor & sequence ion only)
label = df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: x[2:] if isinstance(x, list) else np.nan)
appended_list = []

for r in range(len(label)):
    list_temp = label.loc[r, :].values.tolist()
    list_temp = [x for x in list_temp if isinstance(x, list)]
    list_temp = [item for sublist in list_temp for item in sublist]
    appended_list.append(list_temp)
    
Primary_ion   = pd.Series(appended_list).apply(lambda x: [ion for ion in x if '-' not in ion])
Secondary_ion = pd.Series(appended_list).apply(lambda x: [ion for ion in x if '-' in ion])

# Append a total annotation column
Total_ion = Primary_ion + pd.DataFrame([Primary_ion, Secondary_ion]).T.apply(lambda x: Secondary_filter(x[0], x[1]), axis = 1)

In [ ]:
# Define a function to remove annotations not in the filtered total annotation list
def Annot_filter(peaks, Total_label):
    
    peaks = [peak if (isinstance(peak, list) and len(set(peak[2:]).intersection(Total_label)) > 0) else np.nan 
             for peak in peaks]
    return peaks

In [ ]:
# Filtered peak annotation
all_peaks = df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: x if isinstance(x, list) else np.nan)
appended_list = []

for r in range(len(all_peaks)):
    list_temp = all_peaks.loc[r, :].values.tolist()
    list_temp = [x if isinstance(x, list) else np.nan for x in list_temp]
    appended_list.append(list_temp)

In [ ]:
# Update original dataframe
df_mz_label_uniq_info = df_mz_label_uniq.loc[:,~df_mz_label_uniq.columns.str.contains('peak')]
Ions                  = pd.DataFrame([appended_list, Total_ion]).T.apply(lambda x: Annot_filter(x[0], x[1]), axis = 1)
df_mz_label_uniq      = df_mz_label_uniq_info.join(pd.DataFrame(Ions.tolist(),
                                                           columns = df_mz_label_uniq.filter(like = 'peak').columns))

In [ ]:
# Define a function to retain only unique 43 NL
def uniq_43(labels):
    # If there is only one 43 NL annotation, retain it
    if len(labels) == 1:
        if '43' in labels[0]:
            return labels
        else:
            return np.nan
    
    # In case of multiple annotations
    else:
        # Check whether there is at least one 43 NL annotation
        labels_43 = [('43' in label) for label in labels]
        
        # If there is no 43 NL annotation, return NaN
        if Counter(labels_43)[True] == 0:            
            return np.nan
        
        # If there is at least one 43 NL annotation
        else:
            # If all labels are 43 NL annotations, retain all
            if Counter(labels_43)[True] == len(labels):
                Precur_43 = [label for label in labels if 'Precur' in label]
                seq_43    = [label for label in labels if 
                             any(['y' in label, 'b' in label, 'a' in label]) == True and 
                             len(re.sub(r'[0-9]+', '', label.split('-')[0])) == 1]
                INT_43    = list(set(labels) - set(Precur_43 + seq_43))
                if len(Precur_43) > 0: # If there is precursor 43 NL, return it
                    return Precur_43
                elif len(seq_43) > 0: # If there is no precursor but sequence 43 NL, return it
                    return seq_43
                else: # If there is no precursor & sequence but INT 43 NL, return it
                    return INT_43
            # If there is at least one non-43 NL annotation, 
            else:
                labels_43_non = [label for label in labels if labels_43[labels.index(label)] == False]
                # If non-43 NL is of precursor or sequence ion type, remove 43 NL
                if len([label for label in labels_43_non if 
                        any(['Precur' in label, 'y' in label, 'b' in label, 'a' in label]) == True and 
                        len(re.sub(r'[0-9]+', '', label.split('-')[0])) == 1]) > 0:
                    return np.nan
                # If non-43 NL is an immonium ion, remove 43 NL
                elif len([label for label in labels_43_non if 'IM(' in label]) > 0:
                    return np.nan
                else: # Otherwise, retain 43 NL
                    return list(set(labels) - set(labels_43_non))

In [ ]:
# Define a function to retain only unique INT
def uniq_INT(labels):
    
    INT = [label for label in labels if all(['r' in label, 'IM(' not in label, 'Precur' not in label, 
                                             len(label.split('-')[0]) in range(2,4)]) == True]
    non_INT = list(set(labels) - set(INT))
    # If there is no di/tripeptide, return NaN
    if len(INT) == 0:
        return np.nan
    else:    
        # If there is no non-INT annotation, retain INT annotation
        if len(non_INT) == 0:
            return INT
        else: # If there is at least one non-INT annotation, return NaN
            return np.nan

In [ ]:
# Retain unique 43 NL and INT
unique_NL  = pd.Series(df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: 
                                                                       uniq_43(x[2:])  if isinstance(x, list) else np.nan).values.tolist()).apply(lambda x: [label for label in x if isinstance(label, list)])
unique_INT = pd.Series(df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: 
                                                                       uniq_INT(x[2:]) if isinstance(x, list) else np.nan).values.tolist()).apply(lambda x: [label for label in x if isinstance(label, list)])


In [ ]:
## Unique Cit DI annotations
# Get unique 43 NL annotations
Total_NL_label = unique_NL.apply(lambda x: [label for sublist in x for label in sublist])
df_mz_label_uniq['Total_NL_label'] = Total_NL_label.apply(lambda x: ','.join(x))
df_mz_label_uniq['precNL_label']   = Total_NL_label.apply(lambda x: ','.join([label for label in x if 'Precur' in label]))
df_mz_label_uniq['seqNL_label']    = Total_NL_label.apply(lambda x: ','.join([label for label in x if 
                                                                              any(['y' in label, 'b' in label, 'a' in label]) == True and 
                                                                              len(re.sub(r'[0-9]+', '', label.split('-')[0])) == 1]))
df_mz_label_uniq['intNL_label']    = Total_NL_label.apply(lambda x: ','.join([label for label in x if 
                                                                              all(['r' in label, 'Precur' not in label]) == True]))

In [ ]:
# Get unique INT annotations
Total_INT_label = unique_INT.apply(lambda x: [label for sublist in x for label in sublist])
df_mz_label_uniq['Total_INT_label']  = Total_INT_label.apply(lambda x: ','.join(x))
df_mz_label_uniq['Dipeptide_label']  = Total_INT_label.apply(lambda x: ','.join([label for label in x if 
                                                                                 len(label.split('-')[0]) == 2]))
df_mz_label_uniq['Tripeptide_label'] = Total_INT_label.apply(lambda x: ','.join([label for label in x if 
                                                                                 len(label.split('-')[0]) == 3]))

In [ ]:
## IM(Cit)-NH3 filtering using intensity ratio of IM(R)-NH3/IM(Cit)-NH3
# Get IM(Cit)-NH3 intensity
IMCit_NH3 = pd.Series(df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: x[1] if isinstance(x, list) and 
                                                                      'IM(r)-NH3' in x[2:] else np.nan).values.tolist())
IMCit_NH3 = IMCit_NH3.apply(lambda x: [intensity for intensity in x if pd.notnull(intensity)]).apply(lambda x: x[0] if len(x) > 0 else np.nan)
IMCit_NH3_presence = (IMCit_NH3 > 0).astype(int)

# Get IM(R)-NH3 intensity
IMR_NH3 = pd.Series(df_mz_label_uniq.filter(like = 'peak').applymap(lambda x: x[1] if isinstance(x, list) and 
                                                                    'IM(R)-NH3' in x[2:] else np.nan).values.tolist())
IMR_NH3 = IMR_NH3.apply(lambda x: [intensity for intensity in x if pd.notnull(intensity)]).apply(lambda x: x[0] if len(x) > 0 else np.nan)

# Retain only IM(Cit)-NH3 with intensity greater than that of IM(R)-NH3
IM_filter = ((IMR_NH3/IMCit_NH3) > 1).astype(int)
IMCit_NH3_presence_filtered = IMCit_NH3_presence - (IMCit_NH3_presence*IM_filter)

# Get IM(Cit)-NH3 annotation
df_mz_label_uniq['IM_NH3_label'] = Total_ion.apply(lambda x: ','.join([label for label in x 
                                                                       if label == 'IM(r)-NH3']))*IMCit_NH3_presence_filtered

In [ ]:
# Get unique 43 NL counts
df_mz_label_uniq['Total_NL_count'] = df_mz_label_uniq['Total_NL_label'].apply(lambda x: len(x.split(',')) if len(x) > 0 else 0)
df_mz_label_uniq['precNL_count']   = df_mz_label_uniq['precNL_label'].apply(lambda x: len(x.split(',')) if len(x) > 0 else 0)
df_mz_label_uniq['seqNL_count']    = df_mz_label_uniq['seqNL_label'].apply(lambda x: len(x.split(',')) if len(x) > 0 else 0)
df_mz_label_uniq['intNL_count']    = df_mz_label_uniq['intNL_label'].apply(lambda x: len(x.split(',')) if len(x) > 0 else 0)

In [ ]:
# Get unique INT counts
df_mz_label_uniq['Total_INT_count']  = unique_INT.str.len()
df_mz_label_uniq['Dipeptide_count']  = unique_INT.apply(lambda x: [label for label in x if 
                                                                   len(label[0].split('-')[0]) == 2]).str.len()
df_mz_label_uniq['Tripeptide_count'] = unique_INT.apply(lambda x: [label for label in x if 
                                                                   len(label[0].split('-')[0]) == 3]).str.len()

In [ ]:
# Get IM(Cit)-NH3 counts
df_mz_label_uniq['IM_NH3_count'] = df_mz_label_uniq['IM_NH3_label'].apply(lambda x: 1 if len(x) > 0 else 0)

In [ ]:
## Get intensity coverage (= summed intensity of annotated peaks / total intensity of MS2 peaks)
# Summed intensity of annotated peaks
appended_list = []

for r in range(len(label)):
    list_temp             = label.loc[r, :].values.tolist()
    list_temp             = [1 if x in Total_ion[r] else 0 for x in list_temp]
    appended_list.append(list_temp)
    
annotated_intensity = df_mz_label_uniq.filter(like='peak').applymap(lambda x: x[1] if 
                                                                    isinstance(x, list) else np.nan).sum(axis = 1)

# Total intensity of MS2 peaks in unID spectrum
total_MS2_intensity = df_unID_group['intensity array'].apply(sum)

# Get intensity coverage
df_mz_label_uniq['Intensity_coverage'] = annotated_intensity / total_MS2_intensity

In [ ]:
# Filter out spectra with low intensity coverage
Final_result = df_mz_label_uniq[[col for col in df_mz_label_uniq.columns if 'peak' not in col]]
Final_result_filter = Final_result[Final_result['Intensity_coverage'] >= intensity_coverage].reset_index(drop = True).drop(columns = ['Intensity_coverage'])

# 7. Evaluation of Cit PSMs by logistic regression model

In [ ]:
# Define a function to calculate Cit probability using the EN model developed in the paper
def EN_model(NL_count, INT_count, IM_NH3_count):
    # Regression coefficients
    Intercept = -1.9270
    NL_coeff  = 0.9759
    INT_coeff = 0.6519
    IM_coeff  = 2.7804
    
    # Logit function
    logit = Intercept + NL_coeff*NL_count + INT_coeff*INT_count + IM_coeff*IM_NH3_count
    prob  = np.exp(logit)/(np.exp(logit) + 1)
    
    return prob

In [ ]:
# Apply the EN model
Cit_prob = Final_result_filter[['Total_NL_count', 'Total_INT_count', 'IM_NH3_count']].apply(lambda x: 
                                                                                            EN_model(x['Total_NL_count'], 
                                                                                                     x['Total_INT_count'], 
                                                                                                     x['IM_NH3_count']), 
                                                                                            axis = 1)
Final_result_filter = Final_result_filter.assign(Cit_probability = Cit_prob) # Citrullination status probability
Final_result_filter = Final_result_filter.assign(Cit_prediction = np.where(Cit_prob >= 0.5, 1, 0)) # Citrullination status classification

In [ ]:
# Save result
Final_result_filter.to_csv("False_negative_prediction.csv") # As csv file